In [1]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import nltk
import pysrt, pprint
import gensim
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


C:\Users\Asus\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
#serie = pysrt.open('HIMYM/HIMYM/How.I.Met.Your.Mother.S01E01.DVDRip.XviD-TOPAZ.EnglishHI.srt', encoding='iso-8859-1')

In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
files =  glob.glob("HIMYM/HIMYM/*.srt")

In [4]:
texts = []
for f in files:
    print('Loading '+f)
    subs = pysrt.open(f, encoding='iso-8859-1')
    for s in subs:
        texts.append(s.text)

Loading HIMYM/HIMYM\How.I.Met.Your.Mother.S01E01.DVDRip.XviD-TOPAZ.EnglishHI.srt
Loading HIMYM/HIMYM\How.I.Met.Your.Mother.S01E02.DVDRip.XviD-TOPAZ.EnglishHI.srt
Loading HIMYM/HIMYM\How.I.Met.Your.Mother.S01E03.DVDRip.XviD-TOPAZ.EnglishHI.srt
Loading HIMYM/HIMYM\How.I.Met.Your.Mother.S01E04.DVDRip.XviD-TOPAZ.EnglishHI.srt
Loading HIMYM/HIMYM\How.I.Met.Your.Mother.S01E05.DVDRip.XviD-TOPAZ.EnglishHI.srt
Loading HIMYM/HIMYM\How.I.Met.Your.Mother.S01E06.DVDRip.XviD-TOPAZ.EnglishHI.srt
Loading HIMYM/HIMYM\How.I.Met.Your.Mother.S01E07.DVDRip.XviD-TOPAZ.EnglishHI.srt
Loading HIMYM/HIMYM\How.I.Met.Your.Mother.S01E08.DVDRip.XviD-TOPAZ.EnglishHI.srt
Loading HIMYM/HIMYM\How.I.Met.Your.Mother.S01E09.DVDRip.XviD-TOPAZ.EnglishHI.srt
Loading HIMYM/HIMYM\How.I.Met.Your.Mother.S01E10.DVDRip.XviD-TOPAZ.EnglishHI.srt
Loading HIMYM/HIMYM\How.I.Met.Your.Mother.S01E11.DVDRip.XviD-TOPAZ.EnglishHI.srt
Loading HIMYM/HIMYM\How.I.Met.Your.Mother.S01E12.DVDRip.XviD-TOPAZ.EnglishHI.srt
Loading HIMYM/HIMYM\How.I.Me

In [5]:
print(len(texts))

8570


In [6]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [7]:
seg = []
for t in texts:
    seg.append(word_tokenize(t))

In [8]:
seg

[['OLDER',
  'TED',
  ':',
  'Kids',
  ',',
  'I',
  "'m",
  'gon',
  'na',
  'tell',
  'you',
  'an',
  'incredible',
  'story',
  '.'],
 ['The', 'story', 'of', 'how', 'I', 'met', 'your', 'mother', '.'],
 ['-',
  'Are',
  'we',
  'being',
  'punished',
  'for',
  'something',
  '?',
  '-',
  'No',
  '.'],
 ['-',
  'Yeah',
  ',',
  'is',
  'this',
  'gon',
  'na',
  'take',
  'a',
  'while',
  '?',
  '-',
  'Yes',
  '.'],
 ['Twenty-five',
  'years',
  'ago',
  ',',
  'before',
  'I',
  'was',
  'Dad',
  ',',
  'I',
  'had',
  'this',
  'whole',
  'other',
  'life',
  '.'],
 ['<',
  'i',
  '>',
  'It',
  'was',
  'way',
  'back',
  'in',
  '2005',
  ',',
  '<',
  '/i',
  '>'],
 ['<',
  'i',
  '>',
  'I',
  'was',
  '27',
  ',',
  'just',
  'starting',
  '<',
  '/i',
  '>',
  '<',
  'i',
  '>',
  'to',
  'make',
  'it',
  'as',
  'an',
  'architect',
  '<',
  '/i',
  '>'],
 ['<',
  'i',
  '>',
  'and',
  'living',
  'in',
  'New',
  'York',
  'with',
  'Marshall',
  ',',
  '<',
  '/i',
 

In [9]:
final = []
#remove all tokens that are not alphanumeric
for tokens in seg:
    words = [word.lower() for word in tokens if word.isalpha()]
    print(words[:100])
    final.append(words)

['older', 'ted', 'kids', 'i', 'gon', 'na', 'tell', 'you', 'an', 'incredible', 'story']
['the', 'story', 'of', 'how', 'i', 'met', 'your', 'mother']
['are', 'we', 'being', 'punished', 'for', 'something', 'no']
['yeah', 'is', 'this', 'gon', 'na', 'take', 'a', 'while', 'yes']
['years', 'ago', 'before', 'i', 'was', 'dad', 'i', 'had', 'this', 'whole', 'other', 'life']
['i', 'it', 'was', 'way', 'back', 'in']
['i', 'i', 'was', 'just', 'starting', 'i', 'to', 'make', 'it', 'as', 'an', 'architect']
['i', 'and', 'living', 'in', 'new', 'york', 'with', 'marshall', 'i', 'my', 'best', 'friend', 'from', 'college']
['i', 'my', 'life', 'was', 'good']
['i', 'and', 'then', 'uncle', 'marshall', 'i', 'went', 'and', 'screwed', 'the', 'whole', 'thing', 'up']
['will', 'you', 'marry', 'me']
['yes', 'perfect']
['and', 'then', 'you', 'engaged', 'you', 'pop', 'the', 'champagne', 'you', 'drink', 'a', 'toast']
['you', 'have', 'sex', 'on', 'the', 'kitchen', 'floor']
['do', 'have', 'sex', 'on', 'our', 'kitchen', 'floor

['the', 'following', 'is', 'from', 'the', 'october', 'issue']
['i', 'older', 'ted', 'so', 'natalie', 'and', 'i', 'i', 'started', 'dating', 'again']
['i', 'and', 'just', 'like', 'that', 'it', 'all', 'came', 'back']
['i', 'the', 'tea', 'candles']
['i', 'the', 'sock', 'monkeys']
['i', 'belle', 'and', 'sebastian', 'all', 'of', 'it']
['i', 'it', 'seemed', 'like', 'i', 'was', 'far', 'off']
['well', 'i', 'better', 'run', 'i', 'have', 'my', 'krav', 'maga', 'class', 'in', 'half', 'an', 'hour']
['krav', 'maga', 'how', 'cool', 'is', 'it', 'that', 'she', 'does', 'krav', 'maga']
['thanks', 'honey']
['bye', 'guys', 'bye']
['bye', 'take', 'it', 'easy']
['dude', 'what', 'krav', 'maga']
['i', 'have', 'no', 'idea', 'some', 'kind', 'of', 'yoga']
['you', 'know', 'that', 'natalie', 'she', 'good', 'times']
['yeah', 'she', 'like', 'the', 'best', 'girl', 'you', 'dated', 'in', 'years']
['yeah', 'ted', 'hold', 'on', 'to', 'that', 'one', 'yeah']
['i', 'have', 'to', 'break', 'up', 'with', 'her']
['i', 'do', 'get'

['the', 'lemon', 'law', 'it', 'gon', 'na', 'be', 'a', 'thing']
['possibly', 'starting', 'right', 'now']
['hi', 'katie', 'barney', 'hi', 'it', 'good', 'to', 'finally', 'meet', 'you']
['yeah', 'katie', 'you', 'are', 'about', 'to', 'be', 'a', 'part', 'of', 'history']
['tell', 'your', 'friends']
['it', 'gon', 'na', 'be', 'a', 'thing']
['man', 'ted', 'been', 'acting', 'weird', 'he', 'started', 'labeling', 'all', 'his', 'food']
['he', 'even', 'carved', 'ted', 'into', 'that', 'block', 'of', 'cheese']
['yeah', 'well', 'now', 'it', 'ed']
['i', 'think', 'he', 'not', 'cool', 'with', 'me', 'moving', 'in', 'no', 'that', 'not', 'it']
['i', 'mean', 'you', 'basically', 'lived', 'here', 'all', 'along', 'ted', 'loves', 'you']
['so', 'what', 'he', 'about', 'i', 'do', 'know']
['but', 'when', 'he', 'ready', 'to', 'talk', 'to', 'me', 'about', 'it', 'he', 'come', 'and', 'talk', 'to', 'me', 'about', 'it']
['are', 'you', 'kidding', 'you', 'guys', 'never', 'talk', 'about', 'anything']
['he', 'just', 'let', 'it'

['to', 'get', 'the', 'name', 'of', 'your', 'guest', 'so', 'i', 'could', 'have', 'the', 'place', 'cards', 'printed', 'up']
['did', 'i', 'call', 'you', 'to', 'get', 'the', 'name', 'of', 'your', 'guest']
['is', 'there', 'a', 'place', 'card', 'with', 'her', 'name', 'printed', 'on', 'it']
['she', 'does', 'need', 'a', 'place', 'card', 'she', 'knows', 'her', 'name']
['what', 'she', 'eating']
['you', 'ordered', 'the', 'chicken', 'what', 'did', 'she', 'order']
['do', 'you', 'see', 'how', 'your', 'story', 'is', 'full', 'of', 'holes']
['come', 'on', 'claudia', 'we', 'go', 'way', 'back']
['is', 'there', 'room', 'for', 'just', 'one', 'more', 'person']
['do', 'make', 'me', 'hurt', 'you', 'ted']
['i', 'do', 'believe', 'this', 'claudia', 'crazy']
['but', 'to', 'be', 'fair', 'she', 'also', 'hot']
['i', 'totally', 'checked', 'plus', 'one', 'i', 'sure', 'i', 'did']
['yeah', 'right', 'i', 'did']
['yeah', 'i', 'do', 'think', 'you', 'did', 'you', 'know', 'why']
['because', 'deep', 'down', 'you', 'did', 'wan

['ted', 'what', 'other', 'germany', 'would', 'it', 'be', 'the', 'one', 'in', 'epcot']
['ted', 'sorry']
['congratulations', 'thank', 'you']
['i', 'do', 'know', 'if', 'i', 'gon', 'na', 'go']
['and', 'if', 'you', 'do', 'what', 'does', 'that', 'mean', 'for', 'us']
['i', 'do', 'know']
['i', 'just', 'i', 'do', 'think', 'that', 'really', 'works', 'for', 'anyone']
['oh', 'god', 'no', 'is', 'a', 'lie']
['teenagers', 'tell', 'each', 'other', 'to', 'get', 'laid', 'the', 'summer', 'before', 'college']
['so', 'that', 'leaves', 'us', 'with']
['either', 'you', 'stay', 'here', 'or', 'we', 'break', 'up']
['wow']
['look', 'let', 'just', 'see', 'how', 'things', 'progress', 'and', 'if', 'by', 'september', 'we', 'still']
['i', 'have', 'to', 'leave', 'on', 'monday']
['monday', 'september', 'the', 'ted']
['do', 'you', 'have', 'any', 'thoughts']
['maybe', 'we', 'should', 'each', 'take', 'some', 'time', 'to', 'think', 'about', 'what', 'we', 'want']
['we', 'meet', 'up', 'tomorrow', 'to', 'discuss', 'it']
['it',

['okay', 'but', 'as', 'of', 'right', 'now']
['at', 'you', 'still', 'have', 'a', 'girlfriend']
['ted', 'i', 'love', 'you', 'i', 'love', 'robin']
['but', 'if', 'you', 'do', 'this', 'right', 'now']
['your', 'entire', 'future', 'with', 'her', 'will', 'be', 'built', 'on', 'a', 'crime']
['i', 'just', 'go', 'home', 'ted', 'i', 'do', 'do', 'this', 'the', 'wrong', 'way']
['i', 'hate', 'how', 'you', 'always', 'right']
['it', 'my', 'best', 'and', 'most', 'annoying', 'trait']
['okay']
['i', 'coming', 'home', 'i', 'see', 'you', 'soon']
['hey', 'hey']
['i', 'should', 'probably', 'get', 'going', 'it', 'pretty', 'late']
['i', 'sorry', 'i', 'oh', 'no', 'i', 'understand', 'it', 'fine', 'yeah']
['can', 'i', 'call', 'you', 'tomorrow', 'totally']
['totally', 'call', 'me', 'tomorrow', 'okay']
['good', 'night', 'robin', 'good', 'night', 'ted']
['oh', 'my', 'god', 'we', 'making', 'out']
['right', 'this', 'is', 'crazy']
['right', 'right']
['should', 'we', 'go', 'to', 'my', 'room', 'i', 'mean', 'there', 'five',

['laughs']
['i', 'mean', 'is', 'this', 'not', 'a', 'date', 'i', 'do', 'know']
['so', 'what', 'you', 'actually', 'like', 'this', 'guy', 'now', 'i', 'do', 'know']
['are', 'you', 'gon', 'na', 'hook', 'up', 'with', 'him', 'i', 'do', 'know']
['yes', 'you', 'do', 'no', 'ted', 'i', 'do']
['and', 'you', 'know', 'what', 'that', 'okay']
['i', 'do', 'plan', 'out', 'every', 'second', 'of', 'my', 'life', 'like', 'you', 'do']
['i', 'do', 'plan', 'out', 'every', 'oh', 'really']
['what', 'is', 'all', 'of', 'this']
['i', 'mean', 'why', 'ca', 'you', 'just', 'say', 'hey', 'robin', 'let', 'go', 'get', 'some', 'sushi']
['no', 'it', 'has', 'to', 'be', 'a', 'string', 'quartet', 'and', 'paris', 'and', 'flowers', 'and', 'chocolates']
['and', 'let', 'spend', 'the', 'rest', 'of', 'our', 'lives', 'together']
['exclaims', 'do', 'you', 'think', 'we', 'a', 'little', 'past', 'sushi', 'at', 'this', 'point']
['god', 'you', 'are', 'so', 'terrified', 'of', 'anything', 'real']
['it', 'like', 'you', 'floating', 'out', 'in'

In [10]:
model = gensim.models.Word2Vec(final,batch_words=8,
                               size=200,iter=20,
                               window=10,
                               min_count=1,
                               workers=multiprocessing.cpu_count())

2018-08-14 05:56:00,467 : INFO : collecting all words and their counts
2018-08-14 05:56:00,469 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-14 05:56:00,492 : INFO : collected 4619 word types from a corpus of 63633 raw words and 8570 sentences
2018-08-14 05:56:00,494 : INFO : Loading a fresh vocabulary
2018-08-14 05:56:00,509 : INFO : min_count=1 retains 4619 unique words (100% of original 4619, drops 0)
2018-08-14 05:56:00,511 : INFO : min_count=1 leaves 63633 word corpus (100% of original 63633, drops 0)
2018-08-14 05:56:00,548 : INFO : deleting the raw counts dictionary of 4619 items
2018-08-14 05:56:00,552 : INFO : sample=0.001 downsamples 66 most-common words
2018-08-14 05:56:00,554 : INFO : downsampling leaves estimated 44037 word corpus (69.2% of prior 63633)
2018-08-14 05:56:00,579 : INFO : estimated required memory for 4619 words and 200 dimensions: 9699900 bytes
2018-08-14 05:56:00,584 : INFO : resetting layer weights
2018-08-14 05:56:00,6

2018-08-14 05:56:14,652 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-08-14 05:56:14,654 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-08-14 05:56:14,656 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-08-14 05:56:14,659 : INFO : EPOCH - 14 : training on 63633 raw words (44057 effective words) took 1.2s, 36737 effective words/s
2018-08-14 05:56:15,662 : INFO : EPOCH 15 - PROGRESS: at 92.91% examples, 40854 words/s, in_qsize 8, out_qsize 0
2018-08-14 05:56:15,751 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-08-14 05:56:15,752 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-08-14 05:56:15,752 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-08-14 05:56:15,752 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-08-14 05:56:15,752 : INFO : EPOCH - 15 : training on 63633 raw words (43924 effective words) took 1.1s, 40411 ef

In [11]:
model.save('srt.model')

2018-08-14 05:56:28,643 : INFO : saving Word2Vec object under srt.model, separately None
2018-08-14 05:56:28,647 : INFO : not storing attribute vectors_norm
2018-08-14 05:56:28,650 : INFO : not storing attribute cum_table
2018-08-14 05:56:28,760 : INFO : saved srt.model


In [12]:
model.wv.most_similar('man')

2018-08-14 05:56:40,599 : INFO : precomputing L2-norms of word weight vectors


[('hate', 0.9866960048675537),
 ('enjoy', 0.9847460985183716),
 ('bailing', 0.9842312932014465),
 ('tanya', 0.9841148257255554),
 ('type', 0.9801815152168274),
 ('forgot', 0.9792414903640747),
 ('saying', 0.9789128303527832),
 ('jeez', 0.9787313938140869),
 ('inform', 0.9786931872367859),
 ('embarrassing', 0.9777886867523193)]